In [1]:
import pandas as pd
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pd.read_csv("train.csv", sep='\t')
data.head()

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,замок,1,NaN,NaN,замок владимира мономаха в любече . многочисле...
1,2,замок,1,NaN,NaN,"шильонский замок замок шильйон ( ) , известный..."
2,3,замок,1,NaN,NaN,проведения архитектурно - археологических рабо...
3,4,замок,1,NaN,NaN,"топи с . , л . белокуров легенда о завещании м..."
4,5,замок,1,NaN,NaN,великий князь литовский гедимин после успешной...


### Загрузка готовых эмбендингов

In [4]:
from gensim.models import KeyedVectors
wv = KeyedVectors.load_word2vec_format("ruwikiruscorpora_upos_skipgram_300_2_2018.vec", binary=False)

C:\Users\irush\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Предобработка текста.

Mystem Взята из мануала с сайта с готовыми эмбедингами с целью единообразия разметки по частям речи. Пробовала другие использовать(pymorphy2), но по словарю не сходились наречия, прилагательные ( были прям отдельные формы полная и краткая) и предлоги. Банально другое буквенное обозначение, из-за чего приличная часть слов не категоризировалась бы.
Был ещё один способ с сайта мануала, но не подошл по причине того, что необходима для него Visiual Studia и C++.

In [5]:
import wget

udpipe_url = 'http://rusvectores.org/static/models/udpipe_syntagrus.model'
text_url = 'http://rusvectores.org/static/henry_sobolya.txt'

modelfile = wget.download(udpipe_url)
textfile = wget.download(text_url)

100% [..............................................................................] 25649 / 25649

In [1]:
from pymystem3 import Mystem



ModuleNotFoundError: No module named 'pymystem3'

In [7]:
import requests
import re

url = 'https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map'

mapping = {}
r = requests.get(url, stream=True)
for pair in r.text.split('\n'):
    pair = re.sub('\s+', ' ', pair, flags=re.U).split(' ')
    if len(pair) > 1:
        mapping[pair[0]] = pair[1]

In [8]:
def tag_mystem(text='Текст нужно передать функции в виде строки!'):  
    m = Mystem()
    processed = m.analyze(text)
    tagged = []
    for w in processed:
        try:
            if w["analysis"] != []:
                lemma = w["analysis"][0]["lex"].lower().strip()
                pos = w["analysis"][0]["gr"].split(',')[0]
                pos = pos.split('=')[0].strip()
                if pos in mapping:
                    tagged.append(lemma + '_' + mapping[pos]) # здесь мы конвертируем тэги
                else:
                    tagged.append(lemma + '_X') # на случай, если попадется тэг, которого нет в маппинге
        except KeyError:
            continue # я здесь пропускаю знаки препинания, но вы можете поступить по-другому
    return tagged

#### узнаём сколько контекстов относится к каждому значению каждого слова

In [29]:
data.loc[:,['word', 'gold_sense_id', 'context']].groupby(['word', 'gold_sense_id'])['context'].count()

word   gold_sense_id
бор    1                 14
       2                 42
замок  1                100
       2                 38
лук    1                 65
       2                 45
суда   1                100
       2                 35
Name: context, dtype: int64

In [34]:
data[data['word'] == 'лук']

,context_id,word,gold_sense_id,predict_sense_id,positions,context
138,139,лук,1,NaN,NaN,"они были основными слушателями баллад , а криз..."
139,140,лук,1,NaN,NaN,якутский лук якутский лук принадлежит к северн...
140,141,лук,1,NaN,NaN,"из всех трех типов луков — классики , олимпика..."
141,142,лук,1,NaN,NaN,английский длинный лук английский длинный лук ...
142,143,лук,1,NaN,NaN,металлических утяжелителей дуг . составной лук...
143,144,лук,1,NaN,NaN,"сказании "" есть рассказ о воине , который мог ..."
144,145,лук,1,NaN,NaN,практически не было . реальные боевые свойства...
145,146,лук,1,NaN,NaN,олимпийский лук тетива это тонко натянутая беч...
146,147,лук,1,NaN,NaN,", шербур - октевиль , брест , бордо , байонна ..."
147,148,лук,1,NaN,NaN,"японский длинный лук , , или обычно просто пре..."


Узнаём в каких диапазонах расположены контексты для наших слов.

бор      384:439

замок    1:138

лук      139:248

суда     249:383

каждому слову сопоставляем вектор и для каждого текста ищем сумму слов(векторов). и потом делим на количество векторов. получаем 134 векторов текста 

In [35]:
corpus = np.array([0]*300, dtype=float32)
for text in data['context'][248:383]:
    text_vect = np.array([0]*300, dtype=float32)
    processed_mystem = tag_mystem(text=re.sub(r'[^А-аЯ-я ]', '', str(text)))
    k = 0.
    for word in processed_mystem:
        if word in wv.vocab.keys():
            text_vect += np.array(wv[word])
            k += 1.
    corpus = np.vstack((corpus, text_vect / k))
corpus = corpus[1:]

In [36]:
corpus.shape

(135, 300)

получаем 134 векторов текста для слова "суда"

### Кластеризация для слова "суда"

In [37]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = 2)
kmeans.fit(corpus)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

смотрим как примерно распределены смыслы для слова

In [24]:
kmeans.labels_

array([1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

считаем метрику

In [27]:
from sklearn.metrics import adjusted_rand_score

In [40]:
adjusted_rand_score([1]*100+[0]*35, kmeans.labels_)

0.32101725080207605

### теперь всё тоже самое для слова "замок"

In [41]:
corpus = np.array([0]*300, dtype=float32)
for text in data['context'][0:138]:
    text_vect = np.array([0]*300, dtype=float32)
    processed_mystem = tag_mystem(text=re.sub(r'[^А-аЯ-я ]', '', str(text)))
    k = 0.
    for word in processed_mystem:
        if word in wv.vocab.keys():
            text_vect += np.array(wv[word])
            k += 1.
    corpus = np.vstack((corpus, text_vect / k))
corpus = corpus[1:]

In [47]:
corpus.shape

(138, 300)

получаем 138 векторов текста для слова "замок"

### Кластеризация для слова "замок"

In [50]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = 2)
kmeans.fit(corpus)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

смотрим как примерно распределены смыслы для слова

In [44]:
kmeans.labels_

array([0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

считаем метрику

In [45]:
from sklearn.metrics import adjusted_rand_score

In [48]:
adjusted_rand_score([1]*100+[0]*38, kmeans.labels_)

0.2526680614438896

### теперь всё тоже самое для слова "бор"

In [52]:
corpus = np.array([0]*300, dtype=float32)
for text in data['context'][383:439]:
    text_vect = np.array([0]*300, dtype=float32)
    processed_mystem = tag_mystem(text=re.sub(r'[^А-аЯ-я ]', '', str(text)))
    k = 0.
    for word in processed_mystem:
        if word in wv.vocab.keys():
            text_vect += np.array(wv[word])
            k += 1.
    corpus = np.vstack((corpus, text_vect / k))
corpus = corpus[1:]

In [53]:
corpus.shape

(56, 300)

In [54]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = 2)
kmeans.fit(corpus)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [55]:
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [56]:
from sklearn.metrics import adjusted_rand_score
adjusted_rand_score([0]*14+[1]*42, kmeans.labels_)

1.0

### и наконец-то время лучка

In [57]:
corpus = np.array([0]*300, dtype=float32)
for text in data['context'][138:248]:
    text_vect = np.array([0]*300, dtype=float32)
    processed_mystem = tag_mystem(text=re.sub(r'[^А-аЯ-я ]', '', str(text)))
    k = 0.
    for word in processed_mystem:
        if word in wv.vocab.keys():
            text_vect += np.array(wv[word])
            k += 1.
    corpus = np.vstack((corpus, text_vect / k))
corpus = corpus[1:]

In [58]:
corpus.shape

(110, 300)

In [59]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = 2)
kmeans.fit(corpus)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [60]:
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [61]:
from sklearn.metrics import adjusted_rand_score
adjusted_rand_score([1]*65+[0]*45, kmeans.labels_)

0.9636089083285092

Тем самым среднее значение метрики ARI полуается = (0.32101725080207605 + 0.2526680614438896 + 1.0 + 0.9636089083285092 )/4= 0.634323555